<a href="https://colab.research.google.com/github/DennisMuturi-prog/Grocery_ingredient_generator/blob/main/grocery_list_ingredient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
my_recipe_data=pd.read_csv('/content/drive/MyDrive/archive/recipes_data.csv')

In [4]:
my_recipe_data['ingredients'].head()

,ingredients
0,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva..."
1,"[""1 small jar chipped beef, cut up"", ""4 boned ..."
2,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg..."
3,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans..."
4,"[""1 c. peanut butter"", ""3/4 c. graham cracker ..."


In [8]:
shorter_data=my_recipe_data['ingredients'].iloc[0:500,]
shorter_data

,ingredients
0,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva..."
1,"[""1 small jar chipped beef, cut up"", ""4 boned ..."
2,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg..."
3,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans..."
4,"[""1 c. peanut butter"", ""3/4 c. graham cracker ..."
...,...
495,"[""1/2 c. graham cracker crumbs"", ""1 c. crunchy..."
496,"[""1 lb. ground beef"", ""1/2 lb. ground pork"", ""..."
497,"[""1 (8 oz.) and 1 (3 oz.) pkg. Philadelphia cr..."
498,"[""1 to 1 1/2 lb. ground beef"", ""1/2 c. chopped..."


In [9]:
validation_data=my_recipe_data['ingredients'].iloc[500:700,]

In [20]:

# Path to save the text file
file_path = 'output_validation.txt'

# Writing to the text file
with open(file_path, 'w') as file:
    for row in validation_data:
        # Join the strings with a comma
        row_1= row.replace('[','')
        row_2=row_1.replace('"','')
        paragraph=row_2.replace(']','')
        file.write(paragraph + '\n')  # Write the paragraph followed by a newline

print(f'Data written to {file_path}')
type(shorter_data)
validation_data.head()


Data written to output_validation.txt


,ingredients
500,"[""5 eggs"", ""3 c. milk"", ""1 tsp. mustard"", ""bre..."
501,"[""garlic"", ""onion"", ""celery powder"", ""minced o..."
502,"[""3/4 c. rice (without salt)"", ""1 small can pi..."
503,"[""1 (1 lb.) can salmon"", ""1/2 c. chopped celer..."
504,"[""1/4 c. diced onion"", ""2 packets instant chic..."


In [1]:
import json
f = open('/content/drive/MyDrive/annotations/annotations.json')
TRAIN_DATA = json.load(f)


In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin()

In [23]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.7.5                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-6.1.85+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        en_core_web_sm (3.7.1)        



In [3]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 31/31 [00:00<00:00, 1355.58it/s]


In [4]:
!pip install spacy-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 106.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [5]:
!python -m spacy init fill-config /content/drive/MyDrive/spacyConfig2/base_config.cfg config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [6]:
f2 = open('/content/drive/MyDrive/annotations/annotations (1).json')
VAL_DATA = json.load(f2)

In [7]:
for text, annot in tqdm(VAL_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./val_data.spacy") # save the docbin object

100%|██████████| 11/11 [00:00<00:00, 1737.30it/s]


In [9]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./val_data.spacy --gpu-id 0

ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning

In [11]:
import spacy_transformers
nlp_ner = spacy.load("/content/model-best")

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is d

In [12]:
new_doc=nlp_ner("2 (8 oz.) pkg. cream cheese, softened, 1/2 c. mayonnaise or salad dressing, 8 oz. mild taco sauce, 2 to 4 thinly sliced green onions, 1/2 green pepper, chopped, 1 tomato, chopped, 1 c. Mozzarella cheese, 1 c. Cheddar cheese, 1/2 c. black olives (optional), tortilla chips")

/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


In [13]:
spacy.displacy.render(new_doc, style="ent", jupyter=True)